In [1]:
from typing import Dict

from prefect import Flow, task, case
from prefect import Parameter

from lume_services.results import Result
from lume_services.tasks import (
    configure_lume_services,
    prepare_lume_model_variables,
    check_local_execution,
    SaveDBResult,
    LoadDBResult,
    LoadFile,
    SaveFile,
)
from lume_services.files import TextFile
from lume_model.variables import InputVariable, OutputVariable
from prefect.storage import Module

from lume_lcls_cu_inj_nn.model import LCLSCuInjNN
from lume_lcls_cu_inj_nn import INPUT_VARIABLES, CU_INJ_MAPPING_TABLE

2023-07-31 17:51:54.504659: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
# CONFIGURE LUME-SERVICES
# see https://slaclab.github.io/lume-services/workflows/#configuring-flows-for-use-with-lume-services
configure = configure_lume_services.run()

# CHECK WHETHER THE FLOW IS RUNNING LOCALLY
# If the flow runs using a local backend, the results service will not be available
running_local = check_local_execution.run()
#running_local.set_upstream(configure)

In [30]:
def preprocessing_task(input_variables):
    # scale all values w.r.t. impact factor
    for var_name, var in input_variables.items():

        # if name included in scaling factors
        if (
            CU_INJ_MAPPING_TABLE["impact_name"]
            .str.contains(var_name, regex=False)
            .any()
        ):

            # scaled_val = (
            #     var.value
            #     * CU_INJ_MAPPING_TABLE.loc[
            #         CU_INJ_MAPPING_TABLE["impact_name"] == var_name, "impact_factor"
            #     ].item()
            # )

            var.value = var.value

    return input_variables
def evaluate(formatted_input_vars):

    model = LCLSCuInjNN()

    return model.evaluate(formatted_input_vars)

In [35]:
input = {
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    }

In [45]:
    input_variable_parameter_dict = {
        var_name: Parameter(var_name, default=var.default)
        for var_name, var in INPUT_VARIABLES.items()
    }

    # ORGANIZE INPUT VARIABLE VALUES LUME-MODEL VARIABLES
    formatted_input_vars = prepare_lume_model_variables.run(
        input_variable_parameter_dict, INPUT_VARIABLES
    )

    # Perform scaling of variables
    processed_input_vars = preprocessing_task(formatted_input_vars)

    for key in input.keys():
        processed_input_vars[key].value = input[key]

    # # RUN EVALUATION
    #print(processed_input_vars)
    output_variables = evaluate(processed_input_vars)

    # # SAVE RESULTS TO RESULTS DATABASE, requires LUME-services results backend 
    # with case(running_local, False):
    #     # CREATE LUME-services Result object
    #     formatted_result = format_result(
    #         input_variables=processed_input_vars, output_variables=output_variables
    #     )

    #     # RUN DATABASE_SAVE_TASK
    #     saved_model_rep = save_db_result_task(formatted_result)
    #     saved_model_rep.set_upstream(configure)



[array([[ 4.13000000e-01,  7.49977244e+00,  2.50000000e+02,
         1.70000000e-01, -7.40000000e-03, -7.40000000e-03,
        -8.89970000e+00,  7.00000000e+07,  4.61470020e+00]])]


2023-07-31 18:12:35.944921: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-31 18:12:35.945126: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at matmul_op_impl.h:621 : INTERNAL: cublas error
2023-07-31 18:12:35.945147: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INTERNAL: cublas error
	 [[{{node model_5/dense_60/MatMul}}]]


InternalError: Graph execution error:

Detected at node 'model_5/dense_60/MatMul' defined at (most recent call last):
    File "/home/thakur12/.conda/envs/lume-lcls-cu-inj-nn-torch/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/thakur12/.conda/envs/lume-lcls-cu-inj-nn-torch/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/thakur12/.conda/envs/lume-lcls-cu-inj-nn-torch/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/thakur12/.conda/envs/lume-lcls-cu-inj-nn-torch/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/thakur12/.conda/envs/lume-lcls-cu-inj-nn-torch/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/thakur12/.conda/envs/lume-lcls-cu-inj-nn-torch/lib/py

In [28]:
output_variables

{'distgen:r_dist:sigma_xy:value': ScalarInputVariable(variable_type='scalar', units='mm', parent_variable=None, value_range=[0.2000128411163704, 0.499960029319067], name='distgen:r_dist:sigma_xy:value', value=<Parameter: distgen:r_dist:sigma_xy:value>, precision=None, default=0.413, is_constant=False),
 'distgen:t_dist:length:value': ScalarInputVariable(variable_type='scalar', units='ps', parent_variable=None, value_range=[3.0000935388576626, 11.999451344364768], name='distgen:t_dist:length:value', value=<Parameter: distgen:t_dist:length:value>, precision=None, default=7.499772441611215, is_constant=False),
 'distgen:total_charge:value': ScalarInputVariable(variable_type='scalar', units='pC', parent_variable=None, value_range=[250.0, 250.0], name='distgen:total_charge:value', value=<Parameter: distgen:total_charge:value>, precision=None, default=250.0, is_constant=True),
 'SOL1:solenoid_field_scale': ScalarInputVariable(variable_type='scalar', units='T', parent_variable=None, value_ran